# GRS

In [ ]:
import jinja2
import os
import pandas as pd
#from scipy.optimize import minimize
import statsmodels.api as sm

#%pylab inline

In [ ]:
F = pd.read_stata("C:/Data/Thesis/Factors.dta").set_index("date")

ff1993 = ["rm", "smb", "hml"]
c1997  = ["rm", "smb", "hml", "wml"]
c1997b = ["rm", "smb", "HMLm", "wml"]
ff2015 = ["rm", "smb", "hmlo", "rmw", "cma"]
ff2016 = ["rm", "smb", "hml", "RMWc", "cma"]
b2016  = ["rm", "smb", "HMLm", "wml", "RMWc"]
b2016b = ["rm", "smb", "hml", "wml", "RMWc"]
b2016c = ["rm", "smb", "HMLm", "RMWc"]

In [ ]:
def Sh2(f):
    Ef = pd.np.mean(f, axis=0)
    CoVf = pd.np.cov(f, rowvar=False)
    return Ef.transpose() @ pd.np.linalg.pinv(CoVf) @ Ef

In [ ]:
def GRS(R, F, factors, start="1963-07", end="2016-12"):
    """
    Returns the GRS statistic of Gibbons, Ross and Shanken (1989)
    """

    alphas = []
    errors = []

    for p in R.columns:
        model = sm.OLS(R[start:end][p] - F[start:end]["rf"],
                       sm.add_constant(F[start:end][factors]))
        fit = model.fit()
        alphas.append(fit.params["const"])
        errors.append(fit.resid)

    alphas = pd.np.array(alphas)
    errors = pd.np.array(errors).transpose()

    K = F[factors].shape[1]
    T, N = errors.shape

    Ve = errors.transpose().dot(errors) / (T - K - 1)

    r = F[factors].mean().as_matrix()

    f = F[factors].as_matrix()

    Vf = (f-r).transpose().dot(f-r) / (T - 1)

    Sh2_a = alphas.dot(pd.np.linalg.pinv(Ve)).dot(alphas)
    Sh2_f = r.dot(pd.np.linalg.pinv(Vf)).dot(r)
    Sh2_a, Sh2_f

    return (T/N)*((T-N-K)/(T-K-1))*(Sh2_a/(1+Sh2_f))

In [ ]:
path = "C:/Users/samth/Dropbox/Thesis/Tex/Tables/"

env = jinja2.Environment(
    block_start_string='-%', block_end_string='%-',
    variable_start_string='=%', variable_end_string='%=',
    loader=jinja2.FileSystemLoader(path)
)

## Anomalies

### Without $Size$-$BM^m$-$Prior$

In [ ]:
anomalies = [
    {"name": "$Size$-$BM$",        "path": "C:/Data/FrenchDartmouth/25_Portfolios_5x5.CSV"},
    {"name": "$Size$-$Acc$",       "path": "C:/Data/FrenchDartmouth/25_Portfolios_ME_AC_5x5.csv"},
    {"name": "$Size$-$Beta$",      "path": "C:/Data/FrenchDartmouth/25_Portfolios_ME_BETA_5x5.csv"},
    {"name": "$Size$-$Inv$",       "path": "C:/Data/FrenchDartmouth/25_Portfolios_ME_INV_5x5.CSV"},
    {"name": "$Size$-$NI$",        "path": "C:/Data/FrenchDartmouth/25_Portfolios_ME_NI_5x5.csv"},
    {"name": "$Size$-$OP$",        "path": "C:/Data/FrenchDartmouth/25_Portfolios_ME_OP_5x5.CSV"},
    {"name": "$Size$-$Prior$",     "path": "C:/Data/FrenchDartmouth/25_Portfolios_ME_Prior_12_2.CSV"},
    {"name": "$Size$-$RVar$",      "path": "C:/Data/FrenchDartmouth/25_Portfolios_ME_RESVAR_5x5.csv"},
    {"name": "$Size$-$Var$",       "path": "C:/Data/FrenchDartmouth/25_Portfolios_ME_VAR_5x5.csv"},
    {"name": "$Size$-$BM$-$Inv$",    "path": "C:/Data/FrenchDartmouth/32_Portfolios_ME_BEME_INV_2x4x4.CSV"},
    {"name": "$Size$-$BM$-$OP$",     "path": "C:/Data/FrenchDartmouth/32_Portfolios_ME_BEME_OP_2x4x4.CSV"},
    #{"name": "$Size$-$BM^m$-$Prior$",  "path": "C:/Data/Thesis/32_Portfolios_ME_BM_PR.csv"},
    {"name": "$Size$-$OP$-$Inv$",    "path": "C:/Data/FrenchDartmouth/32_Portfolios_ME_OP_INV_2x4x4.CSV"},
]

for anomaly in anomalies:
    #print(anomaly)
    anomaly["df"] = pd.read_csv(anomaly["path"])
    anomaly["df"] = anomaly["df"].iloc[:642, 1:]
    anomaly["df"].index = F.index
    anomaly["df"] = anomaly["df"].fillna(-99.99)

    anomaly["GRS"] = {
        #"CAPM": "{:.3f}".format(GRS(anomaly["df"], F, ["rm",])),
        #"FF1993": "{:.3f}".format(GRS(anomaly["df"], F, ff1993)),
        "C1997": "{:.2f}".format(GRS(anomaly["df"], F, c1997)),
        "C1997b": "{:.2f}".format(GRS(anomaly["df"], F, c1997b)),
        #"FF2015": "{:.3f}".format(GRS(anomaly["df"], F, ff2015)),
        #"FF2016a": "{:.3f}".format(GRS(anomaly["df"], F, ff2016a)),
        "FF2016": "{:.2f}".format(GRS(anomaly["df"], F, ff2016)),
        "B2016": "{:.2f}".format(GRS(anomaly["df"], F, b2016)),
        "B2016b": "{:.2f}".format(GRS(anomaly["df"], F, b2016b)),
        "B2016c": "{:.2f}".format(GRS(anomaly["df"], F, b2016c)),
    }

All, All_BM, All_Non_BM = {"name": "All"}, {"name": "All ($BM$)"}, {"name": "All (Non-$BM$)"}

for anomaly in anomalies:
    if not "df" in All:
        All["df"] = anomaly["df"]
    All["df"] = All["df"].merge(anomaly["df"], left_index=True, right_index=True)
    if "BM" in anomaly["name"]:
        if not "df" in All_BM:
            All_BM["df"] = anomaly["df"]
        All_BM["df"] = All_BM["df"].merge(anomaly["df"], left_index=True, right_index=True)
    if not "BM" in anomaly["name"]:
        if not "df" in All_Non_BM:
            All_Non_BM["df"] = anomaly["df"]
        All_Non_BM["df"] = All_Non_BM["df"].merge(anomaly["df"], left_index=True, right_index=True)
    
    #anomaly.pop("df")

for anomaly in [All, All_BM, All_Non_BM]:
    anomaly["GRS"] = {
        #"CAPM": "{:.3f}".format(GRS(anomaly["df"], F, ["rm",])),
        #"FF1993": "{:.3f}".format(GRS(anomaly["df"], F, ff1993)),
        "C1997": "{:.2f}".format(GRS(anomaly["df"], F, c1997)),
        "C1997b": "{:.2f}".format(GRS(anomaly["df"], F, c1997b)),
        #"FF2015": "{:.3f}".format(GRS(anomaly["df"], F, ff2015)),
        #"FF2016a": "{:.3f}".format(GRS(anomaly["df"], F, ff2016a)),
        "FF2016": "{:.2f}".format(GRS(anomaly["df"], F, ff2016)),
        "B2016": "{:.2f}".format(GRS(anomaly["df"], F, b2016)),
        "B2016b": "{:.2f}".format(GRS(anomaly["df"], F, b2016b)),
        "B2016c": "{:.2f}".format(GRS(anomaly["df"], F, b2016c)),
    }

    #anomaly.pop("df")

anomalies.extend([All_BM, All_Non_BM, All])

### With $Size$-$BM$-$Prior$

In [ ]:
anomaly = {"name": "$Size$-$BM^m$-$Prior$",  "path": "C:/Data/Thesis/32_Portfolios_ME_BM_PR.csv"}

anomaly["df"] = pd.read_csv(anomaly["path"])
anomaly["df"] = anomaly["df"].iloc[:642, 1:]
anomaly["df"].index = F.index
anomaly["df"] = anomaly["df"].fillna(-99.99)

anomaly["GRS"] = {
    #"CAPM": "{:.3f}".format(GRS(anomaly["df"], F, ["rm",])),
    #"FF1993": "{:.3f}".format(GRS(anomaly["df"], F, ff1993)),
    "C1997": "{:.2f}".format(GRS(anomaly["df"], F, c1997)),
    "C1997b": "{:.2f}".format(GRS(anomaly["df"], F, c1997b)),
    #"FF2015": "{:.3f}".format(GRS(anomaly["df"], F, ff2015)),
    #"FF2016a": "{:.3f}".format(GRS(anomaly["df"], F, ff2016a)),
    "FF2016": "{:.2f}".format(GRS(anomaly["df"], F, ff2016)),
    "B2016": "{:.2f}".format(GRS(anomaly["df"], F, b2016)),
    "B2016b": "{:.2f}".format(GRS(anomaly["df"], F, b2016b)),
    "B2016c": "{:.2f}".format(GRS(anomaly["df"], F, b2016c)),
}

anomalies.append(anomaly)

All, All_BM, All_Non_BM = {"name": "All"}, {"name": "All ($BM$)"}, {"name": "All (Non-$BM$)"}

for anomaly in anomalies:

    print(anomaly["name"], end=" ")
    if anomaly["name"] in ["All", "All ($BM$)", "All (Non-$BM$)"]:
        print("Excluded")
        continue

    print("Included")
    if not "df" in All:
        All["df"] = anomaly["df"]

    All["df"] = All["df"].merge(anomaly["df"], left_index=True, right_index=True)

    if "BM" in anomaly["name"]:
        if not "df" in All_BM:
            All_BM["df"] = anomaly["df"]
        All_BM["df"] = All_BM["df"].merge(anomaly["df"], left_index=True, right_index=True)

    if not "BM" in anomaly["name"]:
        if not "df" in All_Non_BM:
            All_Non_BM["df"] = anomaly["df"]
        All_Non_BM["df"] = All_Non_BM["df"].merge(anomaly["df"], left_index=True, right_index=True)
    
    #anomaly.pop("df")

for anomaly in [All, All_BM, All_Non_BM]:
    anomaly["GRS"] = {
        #"CAPM": "{:.3f}".format(GRS(anomaly["df"], F, ["rm",])),
        #"FF1993": "{:.3f}".format(GRS(anomaly["df"], F, ff1993)),
        "C1997": "{:.2f}".format(GRS(anomaly["df"], F, c1997)),
        "C1997b": "{:.2f}".format(GRS(anomaly["df"], F, c1997b)),
        #"FF2015": "{:.3f}".format(GRS(anomaly["df"], F, ff2015)),
        #"FF2016a": "{:.3f}".format(GRS(anomaly["df"], F, ff2016a)),
        "FF2016": "{:.2f}".format(GRS(anomaly["df"], F, ff2016)),
        "B2016": "{:.2f}".format(GRS(anomaly["df"], F, b2016)),
        "B2016b": "{:.2f}".format(GRS(anomaly["df"], F, b2016b)),
        "B2016c": "{:.2f}".format(GRS(anomaly["df"], F, b2016c)),
    }

    anomaly.pop("df")

anomalies.extend([All_BM, All_Non_BM, All])

In [ ]:
anomalies.append({"name": "$Sh^2$", "GRS": {
    "C1997": "{:.2f}".format(Sh2(F[c1997].as_matrix())),
    "C1997b": "{:.2f}".format(Sh2(F[c1997b].as_matrix())),
    "FF2016": "{:.2f}".format(Sh2(F[ff2016].as_matrix())),
    "B2016": "{:.2f}".format(Sh2(F[b2016].as_matrix())),
    "B2016b": "{:.2f}".format(Sh2(F[b2016b].as_matrix())),
    "B2016c": "{:.2f}".format(Sh2(F[b2016c].as_matrix())),
}})

In [ ]:
template = env.get_template("GRS_template.txt")

In [ ]:
context = {
    "caption": r"GRS Statistic and $Sh^2$ -- 1963-07 through 2016-12",
    "label": "tbl:GRS",
    "model_names": ["C1997", "C1997b", "FF2016", "B2016", "B2016b", "B2016c"],
    "anomalies": anomalies,
}

In [ ]:
with open(os.path.join(path, "GRS_196307_201612.tex"), "w") as table:
    table.write(template.render(context))